In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

FITS_PATH = '../data/fits/PROMISE-Q1-8micron-filled-v0_3.fits'

XSIZE = 1000
YSIZE = 1000
X = 13000
Y = 5000

# Open currimg and display info using memmap to not load the whole image into memory
imageHDU = fits.open(FITS_PATH, memmap=True, mode='denywrite')
imageHDU.info()

# Get display image as Numpy Array called data
data = np.array(imageHDU[0].data[Y:Y+YSIZE, X:X+XSIZE], dtype=np.float32) # type: ignore
imageHDU.close() # Close the file


# Display data
plt.imshow(data, cmap='inferno') #nipy_spectral, inferno, plasma, magma, viridis <- all good


In [ ]:
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

norm = ImageNormalize(stretch=SqrtStretch()) # square root is best ;)
plt.imshow(data, norm=norm, cmap='inferno', interpolation='nearest')

In [ ]:
# plot maybe stars
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
mean, median, std = sigma_clipped_stats(data, sigma=3.0)

daofind = DAOStarFinder(fwhm=3.0, threshold=5.*std)
sources = daofind(data - median)
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
norm = ImageNormalize(stretch=SqrtStretch()) # type: ignore
plt.imshow(data, cmap='inferno', norm=norm)
plt.plot(positions[:, 0], positions[:, 1], 'o', markerfacecolor='none', markeredgecolor='limegreen', markersize=4)
print((mean, median, std))
# these should be removed

In [ ]:
#Gigaofärdig kod 
import copy
copyData = copy.deepcopy(data)-copy.deepcopy(median)
subData = copy.deepcopy(data)-copy.deepcopy(median)
subData[subData > std] = std


# kernel = Gaussian2DKernel(x_stddev=1)

# fixed_img = interpolate_replace_nans(subData, kernel)


norm = ImageNormalize(stretch=SqrtStretch()) # type: ignore


fig = plt.figure(figsize=(20,20))

axs1 = fig.add_subplot(2,2,1)
axs1.imshow(data,cmap='inferno',norm = norm)
axs1.set_title('Original input normalized')
axs2 = fig.add_subplot(2,2,2)
axs2.imshow(copyData,cmap = 'inferno')
axs2.set_title('Subtracted with median')
axs3 = fig.add_subplot(2,2,3)
axs3.imshow(subData, cmap = 'inferno')
axs3.set_title('Interpolated image')

sources = daofind(subData)
if sources:
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    norm = ImageNormalize(stretch=SqrtStretch()) # type: ignore
    # print(positions.size[0])

    axs4 = fig.add_subplot(2,2,4)
    axs4.imshow(data, cmap='inferno', norm=norm)
    plt.plot(positions[:, 0], positions[:, 1], 'o', markerfacecolor='none', markeredgecolor='limegreen', markersize=4)